# テキスト分類タスク

このモジュールでは、**[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** データセットに基づいたシンプルなテキスト分類タスクから始めます。ニュースの見出しを「World（世界）」、「Sports（スポーツ）」、「Business（ビジネス）」、「Sci/Tech（科学/技術）」の4つのカテゴリのいずれかに分類します。

## データセット

データセットを読み込むために、**[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API を使用します。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

データセットのトレーニング部分とテスト部分には、それぞれ `dataset['train']` と `dataset['test']` を使用してアクセスできます。


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


私たちのデータセットから最初の10件の新しい見出しを印刷しましょう。


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## テキストのベクトル化

次に、テキストを**数値**に変換し、テンソルとして表現できるようにする必要があります。単語レベルでの表現を行いたい場合、以下の2つのステップが必要です：

* **トークナイザー**を使用してテキストを**トークン**に分割する。
* それらのトークンの**語彙**を構築する。

### 語彙サイズの制限

AG Newsデータセットの例では、語彙サイズが非常に大きく、10万語以上あります。一般的に言えば、テキストにほとんど出現しない単語は必要ありません — それらは数文にしか現れず、モデルがそれらから学習することはありません。そのため、ベクトライザーのコンストラクタに引数を渡すことで、語彙サイズを小さな数に制限するのが理にかなっています。

これらのステップはどちらも**TextVectorization**レイヤーを使用して処理できます。ベクトライザーオブジェクトをインスタンス化し、その後`adapt`メソッドを呼び出してすべてのテキストを処理し、語彙を構築しましょう：


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **注意** 私たちは、全データセットの一部のみを使用して語彙を構築しています。これは、実行時間を短縮し、待ち時間を減らすためです。しかし、全データセットの中のいくつかの単語が語彙に含まれず、トレーニング中に無視されるリスクを伴います。そのため、全語彙サイズを使用し、`adapt`中に全データセットを処理することで最終的な精度を向上させることは可能ですが、その効果は大きくはありません。

これで実際の語彙にアクセスできるようになりました:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


ベクトライザーを使用すると、任意のテキストを簡単に数値のセットにエンコードできます。


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-wordsテキスト表現

言葉は意味を表すため、時には文中の順序に関係なく、個々の単語を見るだけでテキストの意味を推測できることがあります。例えば、ニュースを分類する際に、*weather*や*snow*といった単語は*天気予報*を示し、*stocks*や*dollar*といった単語は*金融ニュース*に関連する可能性があります。

**Bag-of-words** (BoW)ベクトル表現は、最も理解しやすい伝統的なベクトル表現です。各単語がベクトルのインデックスにリンクされ、ベクトルの要素には、特定の文書内で各単語が出現した回数が含まれます。

![Bag-of-wordsベクトル表現がメモリ内でどのように表現されるかを示す画像。](../../../../../translated_images/ja/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoWは、テキスト内の個々の単語に対するすべてのone-hotエンコードされたベクトルの合計として考えることもできます。

以下は、Scikit LearnのPythonライブラリを使用してBag-of-words表現を生成する例です:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

上記で定義したKerasベクトライザーを使用して、各単語番号をワンホットエンコーディングに変換し、それらのベクトルをすべて加算することもできます。


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **注記**: Kerasの例と結果が異なることに驚くかもしれません。その理由は、Kerasの例ではベクトルの長さが語彙サイズに対応しており、この語彙はAG Newsデータセット全体から構築されています。一方、Scikit Learnの例では、サンプルテキストからその場で語彙を構築しているためです。


## BoW分類器のトレーニング

テキストのバッグオブワーズ表現の作り方を学んだので、それを使う分類器をトレーニングしてみましょう。まず、データセットをバッグオブワーズ表現に変換する必要があります。これは、次のように`map`関数を使うことで実現できます:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

では、1つの線形層を含むシンプルな分類器ニューラルネットワークを定義しましょう。入力サイズは`vocab_size`で、出力サイズはクラス数（4）に対応します。分類タスクを解くため、最終的な活性化関数は**softmax**です。


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


4つのクラスがあるため、80%以上の精度は良い結果と言えます。

## 1つのネットワークとして分類器を訓練する

ベクトライザもKerasのレイヤーであるため、それを含むネットワークを定義し、エンドツーエンドで訓練することができます。この方法では、`map`を使ってデータセットをベクトル化する必要がなく、元のデータセットをそのままネットワークの入力に渡すことができます。

> **Note**: データセット内のフィールド（例えば`title`、`description`、`label`など）を辞書からタプルに変換するために、データセットに対して`map`を適用する必要は依然としてあります。ただし、ディスクからデータを読み込む際に、最初から必要な構造を持つデータセットを構築することが可能です。


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## バイグラム、トライグラム、n-グラム

Bag-of-wordsアプローチの制約の一つは、いくつかの単語が複数の単語からなる表現の一部である場合があることです。例えば、「hot dog」という単語は、他の文脈で使われる「hot」や「dog」という単語とは全く異なる意味を持ちます。「hot」と「dog」を常に同じベクトルで表現すると、モデルが混乱する可能性があります。

これに対処するために、**n-グラム表現**が文書分類の手法でよく使用されます。ここでは、各単語、2語の組み合わせ（バイグラム）、または3語の組み合わせ（トライグラム）の頻度が、分類器を訓練するための有用な特徴となります。例えば、バイグラム表現では、元の単語に加えて、すべての単語ペアを語彙に追加します。

以下は、Scikit Learnを使用してバイグラムのBag-of-Words表現を生成する例です:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-gramアプローチの主な欠点は、語彙サイズが非常に速く増加し始めることです。実際には、n-gram表現を次のユニットで説明する*埋め込み*のような次元削減技術と組み合わせる必要があります。

**AG News**データセットでn-gram表現を使用するには、`TextVectorization`コンストラクタに`ngrams`パラメータを渡す必要があります。バイグラム語彙の長さは**非常に大きく**、私たちの場合では130万以上のトークンがあります！したがって、バイグラムトークンを適切な数に制限するのが理にかなっています。

上記と同じコードを使用して分類器を訓練することもできますが、それは非常にメモリ効率が悪いでしょう。次のユニットでは、埋め込みを使用してバイグラム分類器を訓練します。それまでの間、このノートブックでバイグラム分類器の訓練を試して、より高い精度を得られるかどうか試してみてください。


## BoWベクトルの自動計算

上記の例では、個々の単語のワンホットエンコーディングを合計することで、手動でBoWベクトルを計算しました。しかし、TensorFlowの最新バージョンでは、ベクトライザーのコンストラクタに`output_mode='count'`パラメータを渡すことで、BoWベクトルを自動的に計算することができます。これにより、モデルの定義とトレーニングが大幅に簡単になります。


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## 頻度-逆文書頻度 (TF-IDF)

BoW（Bag of Words）表現では、単語の出現頻度は単語そのものに関係なく同じ方法で重み付けされます。しかし、*a* や *in* のような頻出単語は、専門用語に比べて分類において重要性が低いことは明らかです。ほとんどのNLPタスクでは、ある単語が他の単語よりも重要である場合があります。

**TF-IDF**は、**頻度-逆文書頻度**を意味します。これはBag of Wordsの変形で、単語が文書内に出現するかどうかを示す二値の0/1ではなく、コーパス内での単語の出現頻度に関連する浮動小数点値を使用します。

より正式には、文書$j$内の単語$i$の重み$w_{ij}$は以下のように定義されます：
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ここで、
* $tf_{ij}$ は文書$j$内で単語$i$が出現した回数、つまり以前見たBoW値
* $N$ はコレクション内の文書数
* $df_i$ はコレクション全体で単語$i$を含む文書数

TF-IDF値$w_{ij}$は、単語が文書内で出現する回数に比例して増加し、その単語を含むコーパス内の文書数によって調整されます。これにより、ある単語が他の単語よりも頻繁に出現する事実を補正します。例えば、単語がコレクション内の*すべての*文書に出現する場合、$df_i=N$となり、$w_{ij}=0$となります。この場合、その単語は完全に無視されます。

Scikit Learnを使用すれば、簡単にテキストのTF-IDFベクトル化を作成できます：


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Kerasでは、`TextVectorization`レイヤーに`output_mode='tf-idf'`パラメータを渡すことで、TF-IDF頻度を自動的に計算できます。TF-IDFを使用すると精度が向上するかどうか確認するために、上記で使用したコードを繰り返してみましょう。


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## 結論

TF-IDF表現は異なる単語に頻度の重みを与えることができますが、意味や順序を表現することはできません。有名な言語学者J.R.ファースが1935年に言ったように、「単語の完全な意味は常に文脈的であり、文脈を離れた意味の研究は真剣に受け止められるべきではない。」このコースの後半では、言語モデルを使用してテキストから文脈情報を捉える方法を学びます。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
